# RTEM Hackathon notebook

## Import standard libraries

In [ ]:
import pandas as pd
import numpy as np

## Import RTEM libraries

In [ ]:
# Install onboard client to python, uncomment and run once
'''
!pip install onboard.client
'''

In [ ]:
from onboard.client import RtemClient
from onboard.client.dataframes import points_df_from_streaming_timeseries
from onboard.client.models import PointSelector, TimeseriesQuery, PointData
from datetime import datetime, timezone, timedelta
from typing import List
import pytz

## Load secret

In [ ]:
# Read secret from /secrets/ folder
with open('secrets/secrets.txt') as f:
    secret = f.readlines()[0]

# Setup client
client = RtemClient(api_key=secret)

In [ ]:
all_buildings = list(pd.DataFrame(client.get_all_buildings())['id'])
print(list(pd.DataFrame(client.get_all_buildings())['id'])) # returns list of buildings that you have access to (you may not have 'Laboratory' in your set)
print(len(all_buildings))

In [ ]:
query = PointSelector()
query.point_types     = ['Hot Water Return Temperature']        # can list multiple point
query.equipment_types = ['boiler']     # types, equipment types,
query.buildings       = all_buildings        # buildings, etc.
selection = client.select_points(query)
points = selection["points"]

print(points)

sensor_metadata = pd.DataFrame(client.get_points_by_ids(points))

#start_date = sensor_metadata.last_updated.apply(lambda x: datetime.fromtimestamp(x/1000, timezone.utc)).min()
#end_date = sensor_metadata.first_updated.apply(lambda x: datetime.fromtimestamp(x/1000, timezone.utc)).max()

tz = pytz.timezone('UTC')
start_date = datetime(2018,1,1,0,0,0).replace(tzinfo=tz)
end_date = datetime(2018,12,31,0,0,0).replace(tzinfo=tz)

timeseries_query = TimeseriesQuery(point_ids = selection['points'], start = start_date, end = end_date)
sensor_data = points_df_from_streaming_timeseries(client.stream_point_timeseries(timeseries_query))
sensor_data.head()

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize = (20, 5))
sensor_data.plot(ax=ax, x="timestamp", y=sensor_data.columns.tolist()[1:], title="Hot Water Return Temperature", ylabel="Temperature [F]", rot=90)
#plt.legend(["Supply temperature", "Return temperature"])

In [ ]:
# Output csv and json
# pd.json_normalize(client.get_equipment_types()).to_csv("data/equip_types.csv")
# pd.json_normalize(client.get_all_point_types()).to_csv("data/point_types.csv")
# pd.json_normalize(client.get_all_buildings()).to_csv("data/buildings.csv")
# pd.json_normalize(client.get_all_buildings()).to_json("data/buildings.json")

In [ ]:
"""Ideas"""
"""Use case: Building Doctor"""
# cooling demand/area for each building
# counting the switches on and off for various equipment
# identify indicators for each building which represents deterioration
# benchmarking against other buildings
# build a dashboard

# select equipment
# select indicators
# rank equipment based on specific performance
